In [206]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
from sklearn import preprocessing

In [207]:
#Load Dataset

In [208]:
df = pd.read_csv ('electronics_preprocess.csv')
df=df.dropna(axis=0, how='any', thresh=None, subset=None, inplace=False)

In [209]:
df.head(2)

,item_id,user_id,rating,category,brand
14,1,14,4,Computers & Accessories,HP
42,1,42,5,Computers & Accessories,HP


In [210]:
#charecter encodeing

In [211]:
le = preprocessing.LabelEncoder()
le1=preprocessing.LabelEncoder()
df['labelencoded_category']=le.fit_transform(list(df['category']))
df['labelencoded_Brand']=le1.fit_transform(list(df['brand']))


In [212]:
df.head(2)

,item_id,user_id,rating,category,brand,labelencoded_category,labelencoded_Brand
14,1,14,4,Computers & Accessories,HP,3,16
42,1,42,5,Computers & Accessories,HP,3,16


In [213]:
#pivot for category vs user Id

In [214]:
category_NAME = df.pivot_table(index='category',columns='user_id',values='rating').fillna(0)
category_NAME.head(3)

user_id,14,28,42,62,63,75,76,77,96,104,...,941808,941810,941815,941821,941832,941842,941843,941850,941854,941858
category,,,,,,,,,,,,,,,,,,,,,
Accessories & Supplies,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Camera & Photo,0.0,4.0,0.0,4.0,5.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Car Electronics & GPS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0


In [215]:
#user_input

In [216]:
U_input="Home Audio"

# Part_1: Recomendation of similar product/category.


In [217]:
#procedure

In [218]:

User_input=le.transform([U_input])
User_input

array([5])

In [219]:
#fire user input Query to get the result

In [220]:
query_point=category_NAME.iloc[User_input,:].values.reshape(1, -1)

In [221]:
#prepare unsuperviced learning  model

In [222]:
category_NAME_matrix = csr_matrix(category_NAME.values)
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(category_NAME_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

Findout cosine distance of other categories and indices with respect to the quary point ,
Minimum distance have maximum likelyhood for similar product recomendation

In [223]:
distances, indices = model_knn.kneighbors(query_point, n_neighbors = 6)

In [224]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(category_NAME.index[User_input]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i,category_NAME.index[indices.flatten()[i]], distances.flatten()[i]))
        

Recommendations for Index(['Home Audio'], dtype='object', name='category'):

1: Headphones, with distance of 0.9913037947242741:
2: Computers & Accessories, with distance of 0.9954123435122597:
3: Portable Audio & Video, with distance of 0.9957620192394002:
4: Accessories & Supplies, with distance of 0.9963755189165474:
5: Camera & Photo, with distance of 0.9966187432977444:


#list down similar category

In [225]:
l2=[]
for i in range(0, len(distances.flatten())):
    l1=category_NAME.index[indices.flatten()[i]].split(',')
    l2.append(l1)
    
        

In [226]:
l2 =pd.DataFrame(l2,columns=['category'])


In [227]:
l2 

,category
0,Home Audio
1,Headphones
2,Computers & Accessories
3,Portable Audio & Video
4,Accessories & Supplies
5,Camera & Photo


# Part_2:recomendation of similar brand for particular product/category as well as similar choice of products


In [228]:
#Filter dataset with respect to user input

In [229]:
df1=df[df['labelencoded_category']==User_input[0]]

In [230]:
df1

,item_id,user_id,rating,category,brand,labelencoded_category,labelencoded_Brand
127,7,127,4,Home Audio,Philips,5,32
131,7,131,4,Home Audio,Philips,5,32
134,7,134,5,Home Audio,Philips,5,32
138,7,138,4,Home Audio,Philips,5,32
147,7,147,5,Home Audio,Philips,5,32
...,...,...,...,...,...,...,...
1048472,8375,941768,1,Home Audio,TaoTronics,5,41
1048494,8375,941788,5,Home Audio,TaoTronics,5,41
1048562,1236,941850,5,Home Audio,Pyle,5,35
1048566,827,941854,5,Home Audio,Pyle,5,35


#findout avarage rateing and no of rateing for each brand

In [231]:
Avarage_ratings_brand = pd.DataFrame(df1.groupby('brand')['rating'].mean())
Avarage_ratings_brand ['num of ratings'] = pd.DataFrame(df1.groupby('brand')['rating'].count())


In [232]:
#name of Brand that is associated with user input category

In [233]:
Avarage_ratings_brand

,rating,num of ratings
brand,,
Bose,4.118012,483
EldHus,4.505000,200
Fintie,4.777778,9
Logitech,4.166667,6
Philips,2.475000,40
Pyle,3.831787,3234
Samsung,3.868852,61
Sony,4.488722,532
TaoTronics,4.235576,4177


Prepare pivot  table brand vs  user_id

In [234]:
BRAND_NAME_1 = df1.pivot_table(index='brand',columns='user_id',values='rating').fillna(0)
BRAND_NAME_1.head(3)

user_id,127,131,134,138,147,150,154,157,165,260,...,941181,941477,941478,941492,941756,941768,941788,941850,941854,941858
brand,,,,,,,,,,,,,,,,,,,,,
Bose,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
EldHus,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Fintie,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#input  brand from the list

In [235]:
Input_for_Brand="EldHus"

#procedure

In [236]:
User_input_Brand=le1.transform([Input_for_Brand])

#fire user input Query to get the result

In [237]:
query_point_Brand=BRAND_NAME_1.iloc[User_input_Brand,:].values.reshape(1, -1)

#prepare unsuperviced learning  model

In [238]:
BRAND_NAME_matrix_1 = csr_matrix(BRAND_NAME_1.values)
model_knn_1 = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn_1.fit(BRAND_NAME_matrix_1)

NearestNeighbors(algorithm='brute', metric='cosine')

Findout cosine distance of other categories and indices with respect to the quary point , Minimum distance have maximum likelyhood for similar brand recomendation

In [239]:
distances_1, indices_1 = model_knn_1.kneighbors(query_point_Brand, n_neighbors = 6)

In [240]:
l3=[]
for i in range(0, len(distances_1.flatten())):
    l4=BRAND_NAME_1.index[indices.flatten()[i]].split(',')
    l3.append(l4)

In [241]:
l3 =pd.DataFrame(l3,columns=['brand'])

In [242]:
#list down similar brand

In [243]:
l3

,brand
0,Pyle
1,Philips
2,Logitech
3,Samsung
4,Bose
5,EldHus


In [244]:
choice_of_brand=l3.merge(Avarage_ratings_brand, how='inner', on='brand')
choice_of_brand['category']=U_input


In [245]:
choice_of_brand

,brand,rating,num of ratings,category
0,Pyle,3.831787,3234,Home Audio
1,Philips,2.475000,40,Home Audio
2,Logitech,4.166667,6,Home Audio
3,Samsung,3.868852,61,Home Audio
4,Bose,4.118012,483,Home Audio
5,EldHus,4.505000,200,Home Audio


In [246]:
choice_of_brand.sort_values(by ='rating', ascending = 0)

,brand,rating,num of ratings,category
5,EldHus,4.505000,200,Home Audio
2,Logitech,4.166667,6,Home Audio
4,Bose,4.118012,483,Home Audio
3,Samsung,3.868852,61,Home Audio
0,Pyle,3.831787,3234,Home Audio
1,Philips,2.475000,40,Home Audio


In [247]:
Result_data=l2.merge(choice_of_brand, how='left', on='category')

Final result cointains of brand recomendation as well as category recomendation for particular brand and category

In [248]:
Result_data.sort_values(by ='rating', ascending = 0).fillna("also recomended")

,category,brand,rating,num of ratings
5,Home Audio,EldHus,4.505,200
2,Home Audio,Logitech,4.16667,6
4,Home Audio,Bose,4.11801,483
3,Home Audio,Samsung,3.86885,61
0,Home Audio,Pyle,3.83179,3234
1,Home Audio,Philips,2.475,40
6,Headphones,also recomended,also recomended,also recomended
7,Computers & Accessories,also recomended,also recomended,also recomended
8,Portable Audio & Video,also recomended,also recomended,also recomended
9,Accessories & Supplies,also recomended,also recomended,also recomended
